# Revenue Forecasting Model

This notebook projects Monthly Recurring Revenue (MRR) for the next 6 months based on historical cohort retention curves.

## Key Outputs
1. **Retention Curves** - Visualize how different cohorts retain over time
2. **MRR Forecast** - Project revenue with optimistic, base, and pessimistic scenarios

---

## Setup & Data Loading

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from pathlib import Path

# Configure plot styling
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette("husl")
%matplotlib inline

# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', '{:.2f}'.format)

print("Libraries loaded successfully")

In [ ]:
# Load data
DATA_DIR = Path("../data")

customers = pd.read_csv(DATA_DIR / "customers.csv", parse_dates=["signup_date"])
subscriptions = pd.read_csv(
    DATA_DIR / "subscriptions.csv",
    parse_dates=["start_date", "end_date"]
)

print(f"Loaded {len(customers):,} customers")
print(f"Loaded {len(subscriptions):,} subscriptions")

## Data Overview

Let's examine our customer and subscription data to understand the business context.

In [ ]:
# Customer overview
print("=" * 50)
print("CUSTOMER DATA OVERVIEW")
print("=" * 50)
print(f"\nDate range: {customers['signup_date'].min().date()} to {customers['signup_date'].max().date()}")
print(f"\nCustomers by acquisition channel:")
print(customers['acquisition_channel'].value_counts())

In [ ]:
# Subscription overview
print("=" * 50)
print("SUBSCRIPTION DATA OVERVIEW")
print("=" * 50)
print(f"\nSubscriptions by plan tier:")
print(subscriptions.groupby('plan_tier')['mrr'].agg(['count', 'mean', 'sum']).round(2))
print(f"\nSubscriptions by status:")
print(subscriptions['status'].value_counts())

---

## Cohort Retention Analysis

Retention is the foundation of SaaS revenue forecasting. We calculate retention by:
1. Assigning each customer to a monthly cohort based on signup date
2. Tracking which months each customer had an active subscription
3. Computing the percentage retained at each month since signup

In [ ]:
def calculate_cohort_retention(customers, subscriptions):
    """
    Calculate retention rates by cohort month and month number.
    """
    # Assign customers to cohorts
    customers = customers.copy()
    customers["cohort_month"] = customers["signup_date"].dt.to_period("M")
    
    # Get cohort sizes
    cohort_sizes = customers.groupby("cohort_month")["customer_id"].nunique()
    
    # Generate active months for each subscription
    active_records = []
    for _, sub in subscriptions.iterrows():
        start = sub["start_date"]
        end = sub["end_date"] if pd.notna(sub["end_date"]) else pd.Timestamp.now()
        
        months = pd.period_range(start=start, end=end, freq="M")
        for month in months:
            active_records.append({
                "customer_id": sub["customer_id"],
                "active_month": month
            })
    
    active_df = pd.DataFrame(active_records)
    
    # Merge with cohorts
    active_df = active_df.merge(
        customers[["customer_id", "cohort_month"]],
        on="customer_id"
    )
    
    # Calculate month number
    active_df["month_number"] = (
        active_df["active_month"].astype('int64') - active_df["cohort_month"].astype('int64')
    )
    
    # Aggregate retention
    retention = (
        active_df[active_df["month_number"] >= 0]
        .groupby(["cohort_month", "month_number"])["customer_id"]
        .nunique()
        .reset_index()
    )
    retention.columns = ["cohort_month", "month_number", "retained_customers"]
    
    # Calculate percentages
    retention["cohort_size"] = retention["cohort_month"].map(cohort_sizes)
    retention["retention_pct"] = (
        retention["retained_customers"] / retention["cohort_size"] * 100
    )
    
    return retention

# Calculate retention
retention_df = calculate_cohort_retention(customers, subscriptions)
print(f"Calculated retention for {retention_df['cohort_month'].nunique()} cohorts")

In [ ]:
# Calculate average retention curve
avg_retention = (
    retention_df[retention_df["month_number"] <= 12]
    .groupby("month_number")["retention_pct"]
    .mean()
    .reset_index()
)

print("Average Retention by Month:")
print(avg_retention.to_string(index=False))

### Retention Curves Visualization

The chart below shows retention curves for each cohort (gray lines) and the average across all cohorts (blue line). This visualization helps identify:
- Overall retention health
- Cohort-specific variations
- Points of highest churn risk

In [ ]:
fig, ax = plt.subplots(figsize=(12, 6))

# Plot individual cohorts
for cohort in retention_df["cohort_month"].unique():
    cohort_data = retention_df[retention_df["cohort_month"] == cohort]
    ax.plot(
        cohort_data["month_number"],
        cohort_data["retention_pct"],
        alpha=0.3,
        linewidth=1,
        color="gray"
    )

# Plot average
ax.plot(
    avg_retention["month_number"],
    avg_retention["retention_pct"],
    linewidth=3,
    color="#2563eb",
    label="Average Retention",
    marker="o",
    markersize=6
)

ax.set_xlabel("Months Since Signup", fontsize=12)
ax.set_ylabel("Retention Rate (%)", fontsize=12)
ax.set_title("Cohort Retention Curves", fontsize=14, fontweight="bold")
ax.legend(loc="upper right")
ax.set_ylim(0, 105)
ax.set_xlim(0, 12)

plt.tight_layout()
plt.savefig("output/retention_curves.png", dpi=150, bbox_inches="tight")
plt.show()

---

## MRR Forecasting

We project future MRR using the historical retention curve. The forecast applies month-over-month retention decay to current MRR, creating three scenarios:

- **Optimistic**: Base retention + 10% improvement
- **Base Case**: Historical average retention
- **Pessimistic**: Base retention - 10% decline

In [ ]:
# Calculate current MRR
active_subs = subscriptions[subscriptions["status"] == "active"]
current_mrr = active_subs["mrr"].sum()

print(f"Current MRR: ${current_mrr:,.0f}")
print(f"Active subscriptions: {len(active_subs):,}")
print(f"Average MRR per subscription: ${active_subs['mrr'].mean():.2f}")

In [ ]:
def forecast_mrr(current_mrr, avg_retention, months_ahead=6, scenario_adjustment=0):
    """
    Project MRR for future months using retention decay.
    """
    projections = [current_mrr]
    
    # Calculate month-over-month retention
    retention_rates = avg_retention.set_index("month_number")["retention_pct"].to_dict()
    
    mom_retention = {}
    for month in range(1, max(retention_rates.keys()) + 1):
        if month in retention_rates and (month - 1) in retention_rates:
            if retention_rates[month - 1] > 0:
                mom_retention[month] = retention_rates[month] / retention_rates[month - 1]
            else:
                mom_retention[month] = 0.95
        else:
            mom_retention[month] = 0.95
    
    avg_mom = np.mean(list(mom_retention.values())) if mom_retention else 0.95
    adjusted_retention = min(1.0, max(0.5, avg_mom + (scenario_adjustment / 100)))
    
    mrr = current_mrr
    for _ in range(months_ahead):
        mrr = mrr * adjusted_retention
        projections.append(mrr)
    
    return projections

# Generate forecasts
months_ahead = 6
base = forecast_mrr(current_mrr, avg_retention, months_ahead, 0)
optimistic = forecast_mrr(current_mrr, avg_retention, months_ahead, 10)
pessimistic = forecast_mrr(current_mrr, avg_retention, months_ahead, -10)

print("Forecast generated for 3 scenarios")

### MRR Forecast Visualization

The chart shows projected MRR over the next 6 months. The shaded region represents the range between pessimistic and optimistic scenarios.

In [ ]:
# Create month labels
current_month = datetime.now()
month_labels = [
    (current_month.replace(day=1) + pd.DateOffset(months=i)).strftime("%b %Y")
    for i in range(months_ahead + 1)
]

fig, ax = plt.subplots(figsize=(12, 6))

# Scenario band
ax.fill_between(
    range(months_ahead + 1),
    pessimistic,
    optimistic,
    alpha=0.2,
    color="#2563eb",
    label="Scenario Range"
)

# Individual scenarios
ax.plot(range(months_ahead + 1), optimistic, linewidth=2, linestyle="--",
        color="#16a34a", label="Optimistic (+10% retention)")
ax.plot(range(months_ahead + 1), base, linewidth=3, color="#2563eb",
        label="Base Case", marker="o", markersize=6)
ax.plot(range(months_ahead + 1), pessimistic, linewidth=2, linestyle="--",
        color="#dc2626", label="Pessimistic (-10% retention)")

ax.set_xticks(range(months_ahead + 1))
ax.set_xticklabels(month_labels, rotation=45, ha="right")
ax.set_xlabel("Month", fontsize=12)
ax.set_ylabel("Monthly Recurring Revenue ($)", fontsize=12)
ax.set_title("6-Month MRR Forecast", fontsize=14, fontweight="bold")
ax.legend(loc="upper right")
ax.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f"${x:,.0f}"))

plt.tight_layout()
plt.savefig("output/mrr_forecast.png", dpi=150, bbox_inches="tight")
plt.show()

---

## Executive Summary

In [ ]:
print("=" * 60)
print("REVENUE FORECAST SUMMARY")
print("=" * 60)

print("\n--- Retention Metrics ---")
for month in [1, 3, 6, 12]:
    if month in avg_retention['month_number'].values:
        pct = avg_retention[avg_retention['month_number'] == month]['retention_pct'].values[0]
        print(f"Month {month:2d} Retention: {pct:.1f}%")

print("\n--- MRR Forecast ---")
print(f"Current MRR:        ${current_mrr:,.0f}")
print(f"\n6-Month Projections:")
print(f"  Pessimistic:      ${pessimistic[-1]:,.0f} ({((pessimistic[-1] - current_mrr) / current_mrr) * 100:+.1f}%)")
print(f"  Base Case:        ${base[-1]:,.0f} ({((base[-1] - current_mrr) / current_mrr) * 100:+.1f}%)")
print(f"  Optimistic:       ${optimistic[-1]:,.0f} ({((optimistic[-1] - current_mrr) / current_mrr) * 100:+.1f}%)")

print("\n--- Cohort Overview ---")
print(f"Total Cohorts Analyzed: {retention_df['cohort_month'].nunique()}")

print("\n" + "=" * 60)

---

## Key Takeaways

1. **Retention drives revenue** - Small improvements in monthly retention compound significantly over time
2. **Scenario planning** - The 20% spread between optimistic and pessimistic scenarios highlights the importance of retention initiatives
3. **Cohort analysis** - Understanding which cohorts retain best can inform acquisition strategy

### Recommended Actions
- Focus on Month 1-3 retention (highest churn period)
- Investigate high-performing cohorts to identify success factors
- Implement proactive outreach for customers showing declining engagement